In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import os

In [2]:
events_train0 = pd.read_csv('events_train0.csv')
events_test0 = pd.read_csv('events_test0.csv')

events_train1 = pd.read_csv('events_train1.csv')
events_test1 = pd.read_csv('events_test1.csv')

events_train2 = pd.read_csv('events_train2.csv')
events_test2 = pd.read_csv('events_test2.csv')

In [3]:
events_train = events_train0.append(events_train1).append(events_train2)
events_test = events_test0.append(events_test1).append(events_test2)

events_train.to_csv('events_train.csv')
events_test.to_csv('events_test.csv')

In [4]:
print(events_train.shape)

(111, 75)


In [5]:
print(events_test.shape)

(35, 75)


In [6]:
x_cols = events_train.columns.drop(['YEAR', 'EVENT_ID', 'PATNO', 'Progression', 'hy', 'APPRDX', 'Unnamed: 0'])
events_train_x = events_train[x_cols]
events_test_x = events_test[x_cols]
events_train_y = events_train['Progression']
events_test_y = events_test['Progression']

print(events_train_x.columns)

Index(['Unnamed: 0.1', 'APOE_e4', 'EDUCYRS', 'HISPLAT', 'HVLTFPRL', 'HVLTRDLY',
       'HVLTREC', 'MAPT_cat', 'PD_MED_USE', 'SDMTOTAL', 'SNCA_rs356181_cat',
       'SNCA_rs3910105_cat', 'VLTANIM', 'VLTFRUIT', 'VLTVEG', 'ab_asyn',
       'abeta', 'age', 'age_cat', 'asyn', 'bjlot', 'educ', 'ess', 'ess_cat',
       'fampd_new', 'fampd_old', 'gds', 'gds_cat', 'gen', 'hemohi',
       'hvlt_discrimination', 'hvlt_immediaterecall', 'hvlt_retention', 'lns',
       'moca', 'quip', 'quip_any', 'quip_buy', 'quip_eat', 'quip_gamble',
       'quip_hobby', 'quip_pund', 'quip_sex', 'quip_walk', 'race', 'rem',
       'rem_cat', 'rem_q6', 'rigidity', 'scopa', 'scopa_cv', 'scopa_gi',
       'scopa_pm', 'scopa_sex', 'scopa_therm', 'scopa_ur', 'sft', 'stai',
       'stai_state', 'stai_trait', 'tau', 'tau_asyn', 'urate', 'race_white',
       'race_black', 'race_asian', 'race_other', 'Prediction'],
      dtype='object')


In [7]:
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import roc_auc_score

def select_best_model(train_X, train_y, valid_X, valid_y, C_options, penalty_options):
    best_model, best_C, best_penalty, best_score = None, None, None, 0 
    for C in C_options:
        for penalty in penalty_options:
            model = LogisticRegression(C=C, penalty=penalty).fit(train_X, train_y)
            train_pred = [p[1] for p in model.predict_proba(train_X)]
            train_auc = roc_auc_score(train_y, train_pred)
            valid_pred = [p[1] for p in model.predict_proba(valid_X)]
            valid_auc = roc_auc_score(valid_y, valid_pred)
            print("C=" + str(C) + " | p=" + penalty + " :: train AUC=" + str(train_auc) + " ; valid AUC=" + str(valid_auc))
            if valid_auc > best_score:
                best_model, best_C, best_penalty, best_score = model, C, penalty, valid_auc
    print("Best model: C = " + str(best_C) + ", penalty: " + str(best_penalty))
    return best_model

model = select_best_model(events_train_x, events_train_y, events_test_x, events_test_y, [0.01, 0.1, 0.25, 0.5, 1], ['l1', 'l2'])
test_pred = [p[1] for p in model.predict_proba(events_test_x)]
print("test AUC: ", roc_auc_score(events_test_y, test_pred))


C=0.01 | p=l1 :: train AUC=0.7962962962962963 ; valid AUC=0.7575757575757576
C=0.01 | p=l2 :: train AUC=0.9389978213507626 ; valid AUC=0.7424242424242424
C=0.1 | p=l1 :: train AUC=0.9106753812636166 ; valid AUC=0.8181818181818181
C=0.1 | p=l2 :: train AUC=0.9989106753812637 ; valid AUC=0.6212121212121211
C=0.25 | p=l1 :: train AUC=0.9705882352941178 ; valid AUC=0.7424242424242424
C=0.25 | p=l2 :: train AUC=1.0 ; valid AUC=0.6363636363636364
C=0.5 | p=l1 :: train AUC=0.9934640522875817 ; valid AUC=0.6363636363636364
C=0.5 | p=l2 :: train AUC=1.0 ; valid AUC=0.6363636363636364
C=1 | p=l1 :: train AUC=1.0 ; valid AUC=0.7272727272727273
C=1 | p=l2 :: train AUC=1.0 ; valid AUC=0.6515151515151515
Best model: C = 0.1, penalty: l1
test AUC:  0.8181818181818181


/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  Futur

In [8]:
def sort_features(model, features):
    coef = model.coef_
    feature_coef = [(features[i], coef[0,i]) for i in range(len(features))]
    sorted_feature_coef = sorted(feature_coef, key=lambda x: -x[1]) 
    return sorted_feature_coef

def display_most_predictive_features(sorted_features, n, positive=True): 
    if positive:
        for i in range(n):
            print(sorted_features[i][0] + ": " + str(sorted_features[i][1]))
    else:
        for i in range(len(sorted_features) - 1, len(sorted_features) - 1 - n, -1):
            print(sorted_features[i][0] + ": " + str(sorted_features[i][1]))

feat = sort_features(model, list(events_train_x.columns))
print("MOST POSIVITE ++")
display_most_predictive_features(feat, 10)
print()
print("MOST NEGATIVE --")
display_most_predictive_features(feat, 10, False)
print()
display_most_predictive_features(feat, 66)

MOST POSIVITE ++
bjlot: 0.04820620601324054
age: 0.030322847721765127
PD_MED_USE: 0.01540261313833261
asyn: 0.0012118620841218143
Unnamed: 0.1: 0.0002727152529023506
APOE_e4: 0.0
EDUCYRS: 0.0
HISPLAT: 0.0
HVLTFPRL: 0.0
HVLTRDLY: 0.0

MOST NEGATIVE --
HVLTREC: -0.24539061677573926
abeta: -0.0038807356307881133
urate: -0.0032742879446136324
tau: -0.0014634031320099835
Prediction: 0.0
race_other: 0.0
race_asian: 0.0
race_black: 0.0
race_white: 0.0
tau_asyn: 0.0

bjlot: 0.04820620601324054
age: 0.030322847721765127
PD_MED_USE: 0.01540261313833261
asyn: 0.0012118620841218143
Unnamed: 0.1: 0.0002727152529023506
APOE_e4: 0.0
EDUCYRS: 0.0
HISPLAT: 0.0
HVLTFPRL: 0.0
HVLTRDLY: 0.0
MAPT_cat: 0.0
SDMTOTAL: 0.0
SNCA_rs356181_cat: 0.0
SNCA_rs3910105_cat: 0.0
VLTANIM: 0.0
VLTFRUIT: 0.0
VLTVEG: 0.0
ab_asyn: 0.0
age_cat: 0.0
educ: 0.0
ess: 0.0
ess_cat: 0.0
fampd_new: 0.0
fampd_old: 0.0
gds: 0.0
gds_cat: 0.0
gen: 0.0
hemohi: 0.0
hvlt_discrimination: 0.0
hvlt_immediaterecall: 0.0
hvlt_retention: 0.0
lns: